In [7]:
import optuna
import joblib
import datetime

In [8]:
from stesml.model_tools import train_and_validate_model

In [9]:
def objective(trial):   
    # Save the study before running the next trial
    joblib.dump(study, "../studies/study_" + model_type + "_" + datetime.datetime.now().strftime("%Y%m%d-%H") + ".pkl")
    
    if model_type == 'NN':
        scale = True
        n_layers = trial.suggest_int("n_layers", 1, 5)
        n_hidden_units = trial.suggest_int("n_hidden_units", 10, 100)
        batch_size = trial.suggest_int("batch_size", 1, 100, log=True)
        parameters = {'n_layers': n_layers, 'n_hidden_units': n_hidden_units, 'batch_size': batch_size, 'epochs': 200}
        result, addendum = train_and_validate_model(data_dir, model_type, target, metric, scale, parameters, n_repeats, t_max=360, split_test_data=split_test_data)
    elif model_type == 'XGBoost':
        scale = False
        learning_rate = trial.suggest_float("learning_rate", 0.01, 1)
        num_boost_round = 10000 # Set this as a maximum, model will stop with early stopping
        parameters = {'learning_rate': learning_rate, 'num_boost_round': num_boost_round}
        result, addendum = train_and_validate_model(data_dir, model_type, target, metric, scale, parameters, n_repeats, split_test_data=split_test_data)
    elif model_type == 'RandomForest':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 1, 200, log=True)
        max_depth = trial.suggest_int("max_depth", 1, 100)
        max_samples = trial.suggest_float("max_samples", .01, 1, log=True)
        parameters = {'n_estimators': n_estimators, 'max_depth': max_depth, 'max_samples': max_samples}
        result, addendum = train_and_validate_model(data_dir, model_type, target, metric, scale, parameters, n_repeats, split_test_data=split_test_data)
    
    return result

In [10]:
data_dir = "../data/Sulfur_Models/heating/heating_all"
model_type = 'XGBoost' # Options: NN, XGBoost, RandomForest
target = 'h' # Options: Tavg, h
metric = 'rmse' # Options: rmse, r2
n_repeats = 1 # Number of times to repeat 5-fold CV. Each repeat gives a different shuffle.
split_test_data = True # Split data into train (64%), val (16%), and test (20%) (True) or just train (80%) and val (20%) (False)

if metric == 'rmse':
    direction = 'minimize'
elif metric == 'r2':
    directon = 'maximize'

load_study = False
study_name = "study_XGBoost_20220714-09.pkl"

In [11]:
if load_study:
    study = joblib.load("../studies/" + study_name)
    print("Best trial until now:")
    print(" Value: ", study.best_trial.value)
    print(" Params: ")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")
else:
    study = optuna.create_study(direction=direction)

[I 2022-07-18 12:11:48,992] A new study created in memory with name: no-name-31c265be-d0a1-4334-ac1b-b17b654e83f7


In [12]:
study.optimize(objective, n_trials=250)

[12:12:19] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:97.86226
[20]	test-rmse:24.31489
[40]	test-rmse:8.76030
[60]	test-rmse:6.67400
[80]	test-rmse:6.45699
[100]	test-rmse:6.39986
[120]	test-rmse:6.36710
[140]	test-rmse:6.28952
[160]	test-rmse:6.25675
[180]	test-rmse:6.21325
[200]	test-rmse:6.20491
[220]	test-rmse:6.20457
[228]	test-rmse:6.20474
Split #0, This Result: 6.2047, Average Result: 6.2047
[12:15:21] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually bein

[I 2022-07-18 12:26:45,780] Trial 0 finished with value: 4.353045531223534 and parameters: {'learning_rate': 0.07124237992671427}. Best is trial 0 with value: 4.353045531223534.


Split #4, This Result: 2.9083, Average Result: 4.3530
[12:26:58] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:74.54008
[20]	test-rmse:6.84860
[40]	test-rmse:6.62746
[60]	test-rmse:6.57717
[80]	test-rmse:6.51551
[100]	test-rmse:6.51009
[120]	test-rmse:6.50727
[140]	test-rmse:6.49613
[160]	test-rmse:6.49007
[180]	test-rmse:6.47706
[200]	test-rmse:6.47467
[220]	test-rmse:6.47382
[240]	test-rmse:6.46819
[255]	test-rmse:6.46948
Split #0, This Result: 6.4696, Average Result: 6.4696
[12:30:29] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then 

[I 2022-07-18 12:42:45,308] Trial 1 finished with value: 4.369823234555178 and parameters: {'learning_rate': 0.2982187595588735}. Best is trial 0 with value: 4.353045531223534.


Split #4, This Result: 2.7532, Average Result: 4.3698
[12:42:58] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:39.87043
[20]	test-rmse:6.86044
[40]	test-rmse:6.63881
[60]	test-rmse:6.55759
[80]	test-rmse:6.54751
[100]	test-rmse:6.53834
[120]	test-rmse:6.53755
[140]	test-rmse:6.52918
[160]	test-rmse:6.52601
[180]	test-rmse:6.52194
[200]	test-rmse:6.52384
[203]	test-rmse:6.52381
Split #0, This Result: 6.5238, Average Result: 6.5238
[12:45:51] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or

...

[I 2022-07-18 13:56:44,493] Trial 9 finished with value: 4.358902504523096 and parameters: {'learning_rate': 0.25924658404049106}. Best is trial 0 with value: 4.353045531223534.


Split #4, This Result: 3.0923, Average Result: 4.3589
[13:56:57] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:102.28700
[20]	test-rmse:58.69284
[40]	test-rmse:34.15188
[60]	test-rmse:20.33116
[80]	test-rmse:12.87987
[100]	test-rmse:9.16770
[120]	test-rmse:7.50491
[140]	test-rmse:6.85377
[160]	test-rmse:6.62807
[180]	test-rmse:6.54529
[200]	test-rmse:6.52165
[220]	test-rmse:6.51245
[240]	test-rmse:6.48210
[260]	test-rmse:6.44527
[280]	test-rmse:6.41788
[300]	test-rmse:6.41768
[304]	test-rmse:6.41570
Split #0, This Result: 6.4153, Average Result: 6.4153
[14:01:28] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a fa

[I 2022-07-18 14:37:52,222] Trial 10 finished with value: 4.304502234734034 and parameters: {'learning_rate': 0.028326389959607595}. Best is trial 10 with value: 4.304502234734034.


Split #4, This Result: 3.1214, Average Result: 4.3045
[14:38:04] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:103.75775
[20]	test-rmse:78.71982
[40]	test-rmse:59.95507
[60]	test-rmse:45.74416
[80]	test-rmse:35.08567
[100]	test-rmse:27.08765
[120]	test-rmse:21.00374
[140]	test-rmse:16.65107
[160]	test-rmse:13.43546
[180]	test-rmse:11.16351
[200]	test-rmse:9.57466
[220]	test-rmse:8.47469
[240]	test-rmse:7.74722
[260]	test-rmse:7.29557
[280]	test-rmse:7.00392
[300]	test-rmse:6.84176
[320]	test-rmse:6.75688
[340]	test-rmse:6.67866
[360]	test-rmse:6.62701
[380]	test-rmse:6.59599
[400]	test-rmse:6.58846
[420]	test-rmse:6.56684
[439]	test-rmse:6.57371
Split #0, This Result: 

KeyboardInterrupt: 

In [ ]:
best_params = study.best_params

In [18]:
best_params

{'n_layers': 2, 'n_hidden_units': 79, 'batch_size': 339, 'epochs': 7}

In [21]:
best_value = study.best_value

In [22]:
best_value

1.0137617092656621